In [5]:
import kagglehub

# Download latest version

In [6]:
path = kagglehub.dataset_download("astraszab/facial-expression-dataset-image-folders-fer2013")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\manar\.cache\kagglehub\datasets\astraszab\facial-expression-dataset-image-folders-fer2013\versions\1


In [9]:
from pathlib import Path
path = Path(path)
DATA_DIR = path / "data"

print("FER_ROOT:", path)
print("DATA_DIR:", DATA_DIR)
print("Subfolders at DATA_DIR:", [p.name for p in DATA_DIR.iterdir() if p.is_dir()])

FER_ROOT: C:\Users\manar\.cache\kagglehub\datasets\astraszab\facial-expression-dataset-image-folders-fer2013\versions\1
DATA_DIR: C:\Users\manar\.cache\kagglehub\datasets\astraszab\facial-expression-dataset-image-folders-fer2013\versions\1\data
Subfolders at DATA_DIR: ['test', 'train', 'val']


# FER2013 numeric folder -> name

In [10]:
FER_ID2NAME = {'0':'angry','1':'disgust','2':'fear','3':'happy','4':'sad','5':'surprise','6':'neutral'}

# We keep only these folders

In [11]:
KEEP_IDS = {'3':'happy', '4':'sad', '6':'neutral'}     # folder names in source
NAME2TARGET = {'happy':0, 'neutral':1, 'sad':2}        # our label ids
print("Keeping:", KEEP_IDS)

Keeping: {'3': 'happy', '4': 'sad', '6': 'neutral'}


In [12]:
from torchvision import transforms

IMG_SIZE = 224

train_tfms = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),   # FER2013 is grayscale
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.5,0.5,0.5], [0.5,0.5,0.5]),
])

val_tfms = transforms.Compose([
    transforms.Grayscale(3),
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.5,0.5,0.5], [0.5,0.5,0.5]),
])

In [ ]:
KEEP_IDS     = {'3':'happy', '4':'sad', '6':'neutral'}
NAME2TARGET  = {'happy':0, 'neutral':1, 'sad':2}


ValueError: too many values to unpack (expected 1)